In [2]:
import rasterio
from glob import glob
import subprocess

# Pre-processing

In [3]:
resolution = 50 # meter
rs_method = 'lanczos'

## Litchfield

In [6]:
# Downsample the GT
loc_input = 'data/lidar/litchfield/square_canopy_10m_no_overlap.tif'
loc_output = f'data/lidar/litchfield/square_canopy_{resolution}m_from_10m_{rs_method}.tif'
cmd = f'gdalwarp -tr {resolution} {-resolution} -r {rs_method} -overwrite {loc_input} {loc_output}'
subprocess.call(cmd.split())

# Reproject the coherence image and match the resolution to the GT
src = rasterio.open(loc_output)
epsg = src.crs.to_epsg()

loc_input = 'data/alos2/litchfield_int_coh_20150314_20150326.tif'
loc_output = f'data/alos2/litchfield_int_coh_20150314_20150326_{epsg}.tif'
cmd = f'gdalwarp -t_srs EPSG:{epsg} -tr {resolution} {-resolution} -r {rs_method} -overwrite {loc_input} {loc_output}'
subprocess.call(cmd.split())

# Clip
ulx, lry, lrx, uly = src.bounds
loc_input = f'data/alos2/litchfield_int_coh_20150314_20150326_{epsg}.tif'
loc_output = f'data/alos2/litchfield_int_coh_20150314_20150326_{epsg}_clp.tif'
cmd = f'gdal_translate -projwin {ulx} {uly} {lrx} {lry} {loc_input} {loc_output}'
subprocess.call(cmd.split())

Creating output file that is 102P x 102L.
Processing data/lidar/litchfield/square_canopy_10m_no_overlap.tif [1/1] : 0Using internal nodata values (e.g. -32767) for image data/lidar/litchfield/square_canopy_10m_no_overlap.tif.
Copying nodata values from source data/lidar/litchfield/square_canopy_10m_no_overlap.tif to destination data/lidar/litchfield/square_canopy_50m_from_10m_lanczos.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 1720P x 1681L.
Processing data/alos2/litchfield_int_coh_20150314_20150326.tif [1/1] : 0

Warning 1: data/alos2/litchfield_int_coh_20150314_20150326.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 1720, 1681
0...10...20...30...40...50...60...70...80...90...100 - done.


0

## Albany

In [7]:
# Get meta info from the label
src = rasterio.open(f'data/label/Albany_ht_est.tif')
epsg = src.crs.to_epsg()
xres, yres = src.transform.a, src.transform.e
ulx, lry, lrx, uly = src.bounds

# Match the resolution
loc_input = 'data/alos2/albany_int_coh_20201016_20211015.tif'
loc_output = f'data/alos2/albany_int_coh_20201016_20211015_{epsg}.tif'
cmd = f'gdalwarp -t_srs EPSG:{epsg} -tr {xres} {yres} -r {rs_method} -overwrite {loc_input} {loc_output}'
subprocess.call(cmd.split())

# Adjust the image shape
loc_input = f'data/alos2/albany_int_coh_20201016_20211015_{epsg}.tif'
loc_output = f'data/alos2/albany_int_coh_20201016_20211015_{epsg}_clp.tif'
cmd = f'gdal_translate -projwin {ulx} {uly} {lrx} {lry} -tr {xres} {yres} {loc_input} {loc_output}'
subprocess.call(cmd.split())

Creating output file that is 1706P x 1724L.
Processing data/alos2/albany_int_coh_20201016_20211015.tif [1/1] : 0

Warning 1: data/alos2/albany_int_coh_20201016_20211015.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 1706, 1724
0...10...20...30...40...50...60...70...80...90...100 - done.


0

# DEM Download

In [ ]:
import ee
from IPython.display import clear_output
import time
import json
# from pathlib import Path
# from urllib.request import urlretrieve
# import zipfile

# ee.Authenticate()
ee.Initialize()

In [ ]:
with open('./AOI/litchfield_lidar.geojson','r') as f:
    aoi_json = json.load(f)
    
coords = aoi_json['features'][0]['geometry']['coordinates'][0][0]
aoi = ee.Geometry.Polygon(coords)


# Donload the DEM
img = ee.Image('AU/GA/DEM_1SEC/v10/DEM-H').select(['elevation']).clip(aoi)
task = ee.batch.Export.image.toCloudStorage(image=img,  # an ee.Image object.
                                            description='Download the Australian DEM',
                                            bucket='takahata-dev',   # Must be under the root directory. E.g. "d1/d2" does not work
                                            fileNamePrefix=f'dem/australia_litchfield_30m',
#                                             scale=30, # meter per pixel
                                            # crs='EPSG:28350',
                                            maxPixels=1e+10)

task.start()
while True:
    clear_output(wait=True)
    status = task.status()
    print(status)
    if status['state'] == 'COMPLETED' or status['state'] == 'FAILED':
        break
    time.sleep(2)

In [ ]:
#cmd = 'gsutil cp gs://takahata-dev/sentinel2/2019*NDVI.tif /home/ketak/data_disk/S2/Albany/'
cmd = f'gsutil cp gs://takahata-dev/dem/*litchfield* /home/ketak/data_disk/DEM/'
subprocess.call(cmd.split())